In [0]:
from pyspark.sql.functions import *

In [0]:
#paths
catalog='nyc_taxi'
schema='nyc_schema'
file_name="green_tripdata_2024-01.parquet"

raw_path=f'/Volumes/{catalog}/{schema}/raw_zone/{file_name}'
bronze_path=f'/Volumes/{catalog}/{schema}/bronze_zone'

In [0]:
# reading raw data
df_raw=spark.read.format('parquet').load(raw_path)
display(df_raw)

In [0]:
# adding metadata to bronze data for traceability and debugging
df_bronze=df_raw.withColumn('ingestion_date',current_date()) \
    .withColumn('source_file',df_raw['_metadata.file_path'])
df_bronze.display()

In [0]:
df_bronze.schema

In [0]:
# writing bronze data to bronze zone
df_bronze.write.mode('overwrite').format('delta').saveAsTable('nyc_taxi.nyc_schema.bronze')

In [0]:
%sql
select count(*) from nyc_taxi.nyc_schema.bronze